In [1]:
import sklearn as skl
import statsmodels as sms
import pandas as pd


import bokeh as bk

In [2]:
from sklearn import dummy
from sklearn import ensemble
from sklearn import grid_search
from sklearn import linear_model
from sklearn import cross_validation
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator, ClassifierMixin
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler, MinMaxScaler

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [18]:
data = pd.read_csv("/home/gus/remote_mounts/louise_scratch2/ddrad/phase1/ML_pipeline_out/gen_data_matrix/joined_matrix/data.csv")

In [19]:
bioclim = pd.read_csv('/home/gus/remote_mounts/louise/data/projects/ddrad58/pop_coords_and_bioclim_data.csv')

In [20]:
bioclim

,code,name,lat,long,color,indivs,bio12,bio13,bio14,bio15,bio18,bio19,bio2,bio3,bio6,bio7,bio4,bio5,bio8,bio9
0,KG,Kalangala,-0.323734,32.293724,#FF0000,3,1918,302,79,45,436,306,103,83,153,124,525,277,219,208
1,OT,Otuboi,1.918258,33.302457,#000000,14,1312,194,17,51,134,447,129,79,165,163,1015,328,222,245
2,MS,Masindi,1.683327,31.734009,#0000FF,12,1330,174,33,41,186,388,119,81,166,146,795,312,223,238
3,NB,Natumba,0.836069,33.685820,#00FF00,24,1322,202,40,40,214,310,121,82,166,147,691,313,234,238


In [21]:
# Get column names sections of the data that need to be preprocessed in different ways
snp_cols = [line.strip('\n').replace('\t','_') for line in open('/home/gus/remote_mounts/louise_scratch2/ddrad/phase1/ML_pipeline_out/gen_data_matrix/pruned_by_LD/indep-pairwise.012.pos').readlines()]
bioclim_cols = list(bioclim.columns[6:].values)
gps_cols = ['lat','long']
infection_cols = ['infection']

In [22]:
# bool_to_int ={True:1,False:0}
# data.infection = data.infection.apply(lambda x: bool_to_int[x])

In [23]:
data = data.set_index('individual')

In [24]:
data.head(10)

,Scaffold0_23408,Scaffold0_93384,Scaffold0_206884,Scaffold0_255257,Scaffold0_340643,Scaffold0_356889,Scaffold0_405320,Scaffold0_425425,Scaffold0_500170,Scaffold0_531536,...,bio19,bio2,bio3,bio6,bio7,bio4,bio5,bio8,bio9,infection
individual,,,,,,,,,,,,,,,,,,,,,
MS11_0001,0,0,0,0,0,0,0,0,0,0,...,388,119,81,166,146,795,312,223,238,True
MS11_0004,0,0,0,0,0,0,1,0,0,0,...,388,119,81,166,146,795,312,223,238,True
MS11_0006,0,0,0,0,0,0,0,0,0,0,...,388,119,81,166,146,795,312,223,238,True
MS11_0012,0,0,0,2,0,0,0,2,0,0,...,388,119,81,166,146,795,312,223,238,True
MS11_0013,2,0,0,2,2,0,2,2,2,0,...,388,119,81,166,146,795,312,223,238,False
MS11_0014,0,0,-1,0,0,0,0,0,0,0,...,388,119,81,166,146,795,312,223,238,False
MS11_0015,2,0,0,2,2,0,2,2,2,0,...,388,119,81,166,146,795,312,223,238,True
MS11_0019,0,0,0,-1,-1,0,0,0,0,0,...,388,119,81,166,146,795,312,223,238,True
MS11_0020,0,-1,-1,0,0,0,0,0,0,0,...,388,119,81,166,146,795,312,223,238,False


In [25]:
X_dev, X_eval, y_dev, y_eval = cross_validation.train_test_split(data,
                                                                 data.infection,
                                                                 test_size=0.33,
                                                                 random_state=78534)

In [26]:
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, column=0):
        """Extract a column or set of columns from a pandas.DataFrame
        """

#         assert isinstance(column,(list,int))
        self.column = column

    def fit_transform(self, X, y=None, **kwargs):
        self.fit(X, y, **kwargs)
        return self.transform(X)

    def transform(self, X, **kwargs):
        return X[self.column]
    
    def fit(self, X, y=None, **kwargs):
        return self

In [27]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('extract', ColumnExtractor(data[bioclim_cols + gps_cols])),
            ('scale', Normalizer())
        ])),
        ('factors', Pipeline([
            ('extract', ColumnExtractor(data[snp_cols + infection_cols])),
            ('one_hot', OneHotEncoder())
        ])),
    ])),
#     ('estimators', FeatureUnion([
#         ('knn', ModelTransformer(KNeighborsRegressor(n_neighbors=5))),
#         ('gbr', ModelTransformer(GradientBoostingRegressor())),
#         ('dtr', ModelTransformer(DecisionTreeRegressor())),
#         ('etr', ModelTransformer(ExtraTreesRegressor())),
#         ('rfr', ModelTransformer(RandomForestRegressor())),
#         ('par', ModelTransformer(PassiveAggressiveRegressor())),
#         ('en', ModelTransformer(ElasticNet())),
#         ('cluster', ModelTransformer(KMeans(n_clusters=2)))
#     ])),
    ('clf', RandomForestClassifier())
#     ('clf', dummy.DummyClassifier())
])

In [29]:
pipeline.fit(data,data.infection)

ValueError: Must pass DataFrame with boolean values only

In [16]:
scores = cross_validation.cross_val_score(pipeline, X_dev, y_dev,
                                          cv=3, scoring='roc_auc')
scores

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# pdb